In [1]:
require 'numo/narray'

true

# 70. データの入手・整形
文に関する極性分析の正解データを用い，以下の要領で正解データ（sentiment.txt）を作成せよ．

rt-polarity.posの各行の先頭に"+1 "という文字列を追加する（極性ラベル"+1"とスペースに続けて肯定的な文の内容が続く）
rt-polarity.negの各行の先頭に"-1 "という文字列を追加する（極性ラベル"-1"とスペースに続けて否定的な文の内容が続く）
上述1と2の内容を結合（concatenate）し，行をランダムに並び替える
sentiment.txtを作成したら，正例（肯定的な文）の数と負例（否定的な文）の数を確認せよ．

In [2]:
pos = File.open("../data/rt-polaritydata/rt-polarity.pos").read
neg = File.open("../data/rt-polaritydata/rt-polarity.pos").read

pos_tagged = pos.split("\n").map {|line| "+1 #{line}"}
neg_tagged = neg.split("\n").map {|line| "-1 #{line}"}
sen = (pos_tagged + neg_tagged).shuffle
counter = {"+1" => 0, "-1" => 0}

out = File.open("../data/sentiment.txt", "w")
sen.each do |line|
  out.puts line
end
out.close

open('../data/sentiment.txt').each_line do |line|
  counter[line[0..1]] += 1
end

counter

{"+1"=>5330, "-1"=>5330}

# 71. ストップワード
英語のストップワードのリスト（ストップリスト）を適当に作成せよ．さらに，引数に与えられた単語（文字列）がストップリストに含まれている場合は真，それ以外は偽を返す関数を実装せよ．さらに，その関数に対するテストを記述せよ．

In [3]:
word_freq = Hash.new { |h, k| h[k] = 0 }

open('../data/sentiment.txt').each_line do |line|
  line[2..-1].chomp.split.each do |word|
    word_freq[word] += 1
  end
end

# 出現頻度が高い単語上位100語をストップワードにした
stop_list = word_freq.sort {|(_, v1), (_, v2)| v2 <=> v1}[0..100].map {|arr| arr[0]}
stop_list += word_freq.select {|k, v| v < 5}.map {|arr| arr[0]}

def is_stopped?(word, stop_list)
  stop_list.include? word
end

# goodとかがストップワードになっちゃった...
puts is_stopped?("lion", stop_list)
puts is_stopped?("does", stop_list)
puts is_stopped?("good", stop_list)

false
true
true


# 72. 素性抽出
極性分析に有用そうな素性を各自で設計し，学習データから素性を抽出せよ．素性としては，レビューからストップワードを除去し，各単語をステミング処理したものが最低限のベースラインとなるであろう．



In [4]:
all_words = []
tf_vecs = []
labels = []

File.foreach('../data/sentiment.txt').with_index do |line, index|  
  tf_vec = Hash.new { |h, k| h[k] = 0 }  
  words = line[3..-1].chomp.split.reject {|word| is_stopped?(word, stop_list) }
  all_words += words
  
  words.each do |word|
    tf_vec[word] += 1
  end
  
  labels << line[0..1]
  tf_vecs << tf_vec  
end

all_words.uniq!
word_index = {}

all_words.each_with_index do |word, index|
  word_index.store(word, index)
end

x = Numo::Float64.zeros(tf_vecs.size, all_words.size)
y = Numo::Float64.zeros(tf_vecs.size, 1)
w = Numo::Float64.new(all_words.size, 1).rand

labels.each_with_index do |label, i|
  y[i] = label == '+1' ? 1.0 : 0.0 
end

tf_vecs.each_with_index do |hash, i|
  hash.each do |word, count|
    x[i, word_index[word]] = count
  end
end

1

1

# 73. 学習
72で抽出した素性を用いて、
ロジスティック回帰モデルを学習せよ．

\begin{align}
J(\theta) = - y log(h_{\theta}(x))  - (1 - y) log(1 - h_{ \theta }(x)) \\
h_{\theta} (x) = \frac{1}{e^{-{\theta}^T x}} \\
\frac{\partial J(\theta)}{\partial x} = x^{T} (h_{\theta}(x) - y)
\end{align}

In [33]:
def cost(w, x, y)
  (y * Numo::NMath.log(h(w, x)) - (1-y) * Numo::NMath.log(1 - h(w, x))).mean
end
  

def h(w, x)
  z = x.dot(w)
  1/ (1 + Numo::NMath.exp(-z))
end


def grad(w, x, y)
  x.transpose.dup.dot(h(w, x) - y)
end


def check(w, w_tmp, x, y)
  (cost(w, x, y) - cost(w_tmp, x, y)).abs
end

def train(x, y)
  w = Numo::Float64.new(x.shape[1], 1).rand
  
  100.times do |t|
    w_tmp = w
    
    # 学習率
    eta = 0.01
    w = w - grad(w, x, y) * eta
    
    puts "iter: #{t}, cost: #{cost(w, x, y)}" if t % 10 == 0
    return w if check(w, w_tmp, x, y) < 0.002
  end
  
  return w
end

:train

In [35]:
w = train(x, y)

iter: 0, cost: 1.0211106249407453
iter: 10, cost: 0.07536933179802474
iter: 20, cost: 0.036907298841577584


Numo::DFloat#shape=[4178,1]
[[-0.104363], 
 [0.35492], 
 [0.391686], 
 [-0.25195], 
 [-0.0212675], 
 [0.223804], 
 [-0.0910015], 
 [-0.0326872], 
 [-0.446454], 
 [-0.0230979], 
 [-0.361428], 
 [-0.0235054], 
 [-0.0593691], 
 [0.502511], 
 [-0.238302], 
 [0.656555], 
 [0.301243], 
 [-0.128706], 
 [-0.0670804], 
 [-0.177122], 
 ...

In [36]:
yp = h(w, x) > 0.5

c = 0

y.size.times do |t|
  c += 1 if y[t].to_i == yp[t].to_i
end

10660

In [37]:
c

5330